In [1]:
import pandas as pd
import numpy as np
import sqlite3
from itertools import product
pd.set_option('display.max_columns', 500)

import matplotlib.pyplot as plt

conn = sqlite3.connect(r'C:\Users\Spencer\Environments\aquastaat\project.db')
conn.text_factory = lambda x: str(x, 'latin1')

data = pd.read_sql("SELECT * FROM parameter", conn)

In [2]:
def remove_blanks(data):
    print('Before: ', data.shape)
    data.loc[data.loc[:,'Value']=='','Value'] = None
    data.dropna(axis=0, how='any', inplace=True)
    print('After: ', data.shape)
    
def label_bin(year, year_bins, bin_labels):
    return bin_labels[len(list(filter(lambda y: y < year, year_bins)))-1]
    
def bin_year(data):
    year_bins = [y for y in range(1978, 2020, 5)]
    bin_labels = [l for l in range(1980, 2020, 5)]
    data['YearBin'] = data['Year'].apply(lambda x: label_bin(x, year_bins, bin_labels))

In [3]:
data = data.loc[data['Year']>=1978,:]

In [4]:
remove_blanks(data)
bin_year(data)
data['Value'] = data['Value'].astype(float)

Before:  (92995, 6)
After:  (92898, 6)


In [5]:
data.drop(axis=1, labels=['VariableId', 'AreaId', 'Year'], inplace=True)

In [6]:
All = slice(None)
WSI = 'SDG 6.4.2. Water Stress'
TFW = 'Total freshwater withdrawal (primary and secondary)'

In [7]:
tfw_predictors = [
    'Total population',
    'Cultivated area (arable land + permanent crops)',
    'Gross Domestic Product (GDP)',
    'Total renewable surface water',
]

In [8]:
data = data.pivot_table(index=['Area', 'YearBin'], columns='VariableName', values='Value', aggfunc=np.mean)

In [9]:
data = data.loc[:, tfw_predictors + [WSI, TFW]]

In [10]:
data

VariableName         Total population  \
Area        YearBin                     
Afghanistan 1980             12750.00   
            1985             11503.00   
            1990             13981.00   
            1995             18382.00   
            2000             21980.00   
            2005             26617.00   
            2010             30697.00   
            2015             35530.00   
Albania     1980              2788.00   
            1985              3121.00   
            1990              3241.00   
            1995              3093.00   
            2000              3119.00   
            2005              3024.00   
            2010              2920.00   
            2015              2930.00   
Algeria     1980             20576.00   
            1985             23918.00   
            1990             27181.00   
            1995             29887.00   
            2000             31995.00   
            2005             34300.00   
            2010             37566.00   
            2015             41318.00   
Andorra     1980                39.11   
            1985                48.46   
            1990                58.89   
            1995                64.33   
            2000                70.05   
            2005                82.68   
...                               ...   
Viet Nam    1990             71130.00   
            1995             77453.00   
            2000             81956.00   
            2005             85890.00   
            2010             90452.00   
            2015             95541.00   
Yemen       1980              8765.00   
            1985             10629.00   
            1990             13326.00   
            1995             16409.00   
            2000             18919.00   
            2005             21752.00   
            2010             24910.00   
            2015             28250.00   
Zambia      1980              6306.00   
            1985              7386.00   
            1990              8452.00   
            1995              9667.00   
            2000             11120.00   
            2005             12726.00   
            2010             14700.00   
            2015             17094.00   
Zimbabwe    1980              7722.00   
            1985              9293.00   
            1990             10683.00   
            1995             11710.00   
            2000             12501.00   
            2005             13330.00   
            2010             14711.00   
            2015             16530.00   

VariableName         Cultivated area (arable land + permanent crops)  \
Area        YearBin                                                    
Afghanistan 1980                                             8054.00   
            1985                                             8045.00   
            1990                                             8030.00   
            1995                                             7790.00   
            2000                                             7753.00   
            2005                                             7910.00   
            2010                                             7910.00   
            2015                                             7910.00   
Albania     1980                                              709.00   
            1985                                              714.00   
            1990                                              703.00   
            1995                                              700.00   
            2000                                              699.00   
            2005                                              698.00   
            2010                                              696.00   
            2015                                              703.50   
Algeria     1980                                             7509.00   
            1985                                

In [11]:
areas = data.index.levels[0]
bin_labels = [l for l in range(1980, 2020, 5)]
training_data = pd.DataFrame(list(product(areas, bin_labels)), columns=['Area', 'YearBin'])
training_data.set_index(['Area', 'YearBin'], inplace=True)
training_data = training_data.join(data, how='left')
training_data = training_data.groupby('Area').apply(lambda x: x.interpolate(limit_direction='both'))
training_data.dropna(axis=0, inplace=True)

In [16]:
training_data.to_csv('training_data.csv')

In [15]:
training_data

Total population  \
Area        YearBin                     
Afghanistan 1980              12750.0   
            1985              11503.0   
            1990              13981.0   
            1995              18382.0   
            2000              21980.0   
            2005              26617.0   
            2010              30697.0   
            2015              35530.0   
Albania     1980               2788.0   
            1985               3121.0   
            1990               3241.0   
            1995               3093.0   
            2000               3119.0   
            2005               3024.0   
            2010               2920.0   
            2015               2930.0   
Algeria     1980              20576.0   
            1985              23918.0   
            1990              27181.0   
            1995              29887.0   
            2000              31995.0   
            2005              34300.0   
            2010              37566.0   
            2015              41318.0   
Angola      1980               9582.0   
            1985              11218.0   
            1990              12968.0   
            1995              15089.0   
            2000              17573.0   
            2005              20998.0   
...                               ...   
Viet Nam    1990              71130.0   
            1995              77453.0   
            2000              81956.0   
            2005              85890.0   
            2010              90452.0   
            2015              95541.0   
Yemen       1980               8765.0   
            1985              10629.0   
            1990              13326.0   
            1995              16409.0   
            2000              18919.0   
            2005              21752.0   
            2010              24910.0   
            2015              28250.0   
Zambia      1980               6306.0   
            1985               7386.0   
            1990               8452.0   
            1995               9667.0   
            2000              11120.0   
            2005              12726.0   
            2010              14700.0   
            2015              17094.0   
Zimbabwe    1980               7722.0   
            1985               9293.0   
            1990              10683.0   
            1995              11710.0   
            2000              12501.0   
            2005              13330.0   
            2010              14711.0   
            2015              16530.0   

                     Cultivated area (arable land + permanent crops)  \
Area        YearBin                                                    
Afghanistan 1980                                              8054.0   
            1985                                              8045.0   
            1990                                              8030.0   
            1995                                              7790.0   
            2000                                              7753.0   
            2005                                              7910.0   
            2010                                              7910.0   
            2015                                              7910.0   
Albania     1980                                               709.0   
            1985                                               714.0   
            1990                                               703.0   
            1995                                               700.0   
            2000                                               699.0   
            2005                                               698.0   
            2010                                               696.0   
            2015                                               703.5   
Algeria     1980                                              7509.0   
            1985                                              7624.0 